# Create your own database and fill it with data
#### Log in to the virtual machine using SSH
`vagrant ssh couse`
#### Add permissions to write to your folder
`sudo chown vagrant:vagrant /home/vagrant/mynotebooks -R`

#### Go to the shared directory
`cd /home/vagrant/mynotebooks`

#### Create Django project
`django-admin startproject myproj1`

`cd myproj1`

#### Create first App (place to store your models)
`django-admin startapp myapp1`


#### Edit notebooks/myproj1/myapp1/models.py

#### Define your models

In [2]:
from django.db import models

class Profile(models.Model):
    ''' Description of T/S Profile'''
    lon = models.FloatField()
    lat = models.FloatField()

    
class Variable(models.Model):
    ''' Description of Temperature, Salinity, etc '''
    name = models.CharField(max_length=200)
    units = models.CharField(max_length=200)

    
class Measurement(models.Model):
    ''' Description of a measurement '''
    variable = models.ForeignKey(Variable)
    profile = models.ForeignKey(Profile)
    depth = models.FloatField()
    value = models.FloatField()

### Edit `mynotebooks/myproj1/myproj1/settings.py`

In [ ]:
INSTALLED_APPS = (
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'myapp1',                        ### ADD THIS LINE. RESPECT SYNTAX!
)

## Create database
```
cd ~/mynotebooks/myproj1
spatialite db.sqlite3 "SELECT InitSpatialMetaData();"

```

## Migrate your models to your database
```
./manage.py makemigrations
./manage.py migrate
```

## Use the database in IPython Notebook
### Create a Notebook INSIDE `mynotebooks/myproj1`
### Start up Django:

In [2]:
import os, sys
print sys.path.append('/home/vagrant/mynotebooks/myproj1/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproj1.settings")
import django
django.setup()

from myapp1.models import Profile, Variable, Measurement

%whos

None
Variable      Type         Data/Info
------------------------------------
Measurement   ModelBase    <class 'myapp1.models.Measurement'>
Profile       ModelBase    <class 'myapp1.models.Profile'>
Variable      ModelBase    <class 'myapp1.models.Variable'>
django        module       <module 'django' from '/u<...>ges/django/__init__.pyc'>
os            module       <module 'os' from '/usr/lib/python2.7/os.pyc'>
sys           module       <module 'sys' (built-in)>


### Use your models. Create a profile, a variable and a measurement

In [3]:
# create profile and add coordinates
p = Profile()
p.lon = 10
p.lat = 20
p.save()  # save into database

# variable and add name, units
v = Variable()
v.name='salinity'
v.units='psu'
v.save()


# create a measurement
m = Measurement()
m.variable = v
m.profile = p
m.depth = 10
m.value = 35
m.save()


# Use your imagination
# Create many more measurements

# get all measurements
ms = Measurement.objects.all()
print ms
print ms[0].value
print ms[0].profile.lon
print ms[0].variable.name

# find only salinities
salOnly = Measurement.objects.filter(variable__name='salinity')

# find only measurements deeper than 5 m
deepMes =  Measurement.objects.filter(depth__gte=5)

[<Measurement: Measurement object>, <Measurement: Measurement object>]
35.0
10.0
salinity
